In [1]:
import numpy as np
from lightfm.datasets import fetch_movielens
data = fetch_movielens(min_rating=5.0)

/Users/YINAN/anaconda3/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [16]:
from lightfm import LightFM

/Users/YINAN/anaconda3/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [21]:
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=100, num_threads=2)

CPU times: user 1.22 s, sys: 2.99 ms, total: 1.22 s
Wall time: 1.22 s


In [17]:
from lightfm.evaluation import precision_at_k

In [23]:
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

Train precision: 0.43
Test precision: 0.04


# Test it with my own data set

In [1]:
import os
import sys
import pandas as pd
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
from support.utility import read_data
from support.utility import train_test_split

In [3]:
data = read_data(sample_frac=0.9)
(train, test) = train_test_split(data)

In [5]:
print(train.shape)
print(test.shape)

(18418865, 2)
(146080, 2)


In [11]:
from algorithms.CollaborativeFilterItem import CollaborativeFilterItem

In [12]:
cfi = CollaborativeFilterItem()
cfi.setup(train)
train_urm = cfi.urm
cfi.setup(test)
test_urm = cfi.urm

In [13]:
train_urm

<28422x62123 sparse matrix of type '<class 'numpy.int64'>'
	with 582571 stored elements in Compressed Sparse Row format>

In [14]:
test_urm

<29216x62986 sparse matrix of type '<class 'numpy.int64'>'
	with 146080 stored elements in Compressed Sparse Row format>

In [27]:
model = LightFM(loss='bpr')
%time model.fit(test_urm,item_features=track_item_matrix,epochs=20, num_threads=2)

CPU times: user 3.09 s, sys: 13 ms, total: 3.1 s
Wall time: 3.1 s


In [29]:
print("Train precision: %.2f" % precision_at_k(model, test_urm, k=5, item_features=track_item_matrix).mean())

Train precision: 0.00


In [23]:
track_item_matrix = cfi.generate_track_artist_matrix(1)

* generating track_artist_matrix with weight 1.0


In [26]:
track_item_matrix = track_item_matrix.tocsr()